# Stable Diffusion WebUI Manager

Notebook này giúp bạn quản lý và tương tác với Stable Diffusion WebUI trên SageMaker.

## 1. Kiểm tra cài đặt

In [ ]:
# Kiểm tra GPU
!nvidia-smi

In [ ]:
# Kiểm tra cài đặt PyTorch
!pip list | grep -E "torch|xformers"

In [ ]:
# Kiểm tra thư mục cài đặt
!ls -la /home/ec2-user/SageMaker/stable-diffusion-webui

## 2. Kiểm tra log cài đặt

In [ ]:
# Xem log cài đặt
!tail -n 50 /home/ec2-user/SageMaker/install.log

## 3. Khởi động WebUI

In [ ]:
import subprocess
import os
import time
import socket
from IPython.display import display, HTML

def is_port_in_use(port):
    """Kiểm tra xem port có đang được sử dụng không"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('localhost', port)) == 0

def start_webui():
    """Khởi động Stable Diffusion WebUI"""
    webui_dir = "/home/ec2-user/SageMaker/stable-diffusion-webui"
    port = 8888
    
    # Kiểm tra xem WebUI đã chạy chưa
    if is_port_in_use(port):
        display(HTML(f'''
        <div style="background-color: #d4edda; color: #155724; padding: 15px; border-radius: 5px; margin: 10px 0;">
            <h3 style="margin-top: 0;">✅ WebUI đã đang chạy!</h3>
            <p>Truy cập WebUI tại URL: <a href="/proxy/8888/" target="_blank">/proxy/8888/</a></p>
        </div>
        '''))
        return
    
    # Kiểm tra xem script khởi động có tồn tại không
    launch_script = f"{webui_dir}/launch_webui.sh"
    if not os.path.exists(launch_script):
        display(HTML(f'''
        <div style="background-color: #f8d7da; color: #721c24; padding: 15px; border-radius: 5px; margin: 10px 0;">
            <h3 style="margin-top: 0;">❌ Lỗi: Script khởi động không tồn tại!</h3>
            <p>Vui lòng đợi quá trình cài đặt hoàn tất hoặc kiểm tra log cài đặt.</p>
        </div>
        '''))
        return
    
    # Khởi động WebUI
    display(HTML(f'''
    <div style="background-color: #cce5ff; color: #004085; padding: 15px; border-radius: 5px; margin: 10px 0;">
        <h3 style="margin-top: 0;">🚀 Đang khởi động WebUI...</h3>
        <p>Quá trình này có thể mất 2-3 phút. Vui lòng đợi.</p>
    </div>
    '''))
    
    # Chạy script trong background
    process = subprocess.Popen(
        ["bash", launch_script],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        cwd=webui_dir
    )
    
    # Đợi WebUI khởi động
    max_wait = 180  # 3 phút
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        if is_port_in_use(port):
            display(HTML(f'''
            <div style="background-color: #d4edda; color: #155724; padding: 15px; border-radius: 5px; margin: 10px 0;">
                <h3 style="margin-top: 0;">✅ WebUI đã sẵn sàng!</h3>
                <p>Truy cập WebUI tại URL: <a href="/proxy/8888/" target="_blank">/proxy/8888/</a></p>
            </div>
            '''))
            return
        time.sleep(5)
    
    # Nếu quá thời gian mà vẫn chưa khởi động được
    display(HTML(f'''
    <div style="background-color: #f8d7da; color: #721c24; padding: 15px; border-radius: 5px; margin: 10px 0;">
        <h3 style="margin-top: 0;">⚠️ Khởi động quá thời gian</h3>
        <p>WebUI có thể vẫn đang khởi động. Vui lòng đợi thêm hoặc kiểm tra log.</p>
        <p>Thử truy cập: <a href="/proxy/8888/" target="_blank">/proxy/8888/</a></p>
    </div>
    '''))

In [ ]:
# Khởi động WebUI
start_webui()

## 4. Hướng dẫn sử dụng Inpainting với ControlNet

### Bước 1: Chọn model

1. Trong giao diện WebUI, tìm dropdown model
2. Chọn: `sd1_5-epiCRealism.safetensors`

### Bước 2: Chuyển đến tab Inpaint

1. Click vào tab "img2img"
2. Chọn sub-tab "Inpaint"

### Bước 3: Cấu hình prompt

1. Trong trường prompt, viết mô tả cho hình ảnh đầu ra mong muốn
2. Ví dụ: "a beautiful landscape with mountains and lake, high quality, detailed"

### Bước 4: Upload và mask hình ảnh

1. Click "Upload" để tải lên hình ảnh gốc
2. Sử dụng công cụ brush để tô lên các vùng bạn muốn mở rộng/chỉnh sửa
3. Tô dọc theo các cạnh nơi bạn muốn hình ảnh được mở rộng

### Bước 5: Cấu hình Inpaint

1. **Resize mode**: Chọn "Resize and fill"
2. **Mask mode**: Chọn "Inpaint masked"
3. **Denoising strength**: Điều chỉnh thanh trượt (giá trị cao = nhiều thay đổi, giá trị thấp = gần với bản gốc)

### Bước 6: Bật ControlNet

1. Cuộn xuống để tìm phần "ControlNet"
2. Đánh dấu vào ô "Enable"

### Bước 7: Cấu hình ControlNet

1. **Upload control image**: Click "Upload independent control image"
2. Tải lên cùng hình ảnh bạn đã sử dụng cho inpainting (hoặc một hình ảnh điều khiển khác)
3. **Control type**: Chọn "Inpaint"
4. **Model**: Chọn `Control_v11p_sd15_inpaint` từ dropdown
5. **Control mode**: Chọn "ControlNet is more important"
6. **Resize mode**: Chọn "Resize and fill"

### Bước 8: Tạo hình ảnh

1. Click nút "Generate"
2. Đợi quá trình xử lý hoàn tất
3. Xem kết quả trong panel output

## 5. Quản lý tài nguyên

In [ ]:
# Kiểm tra dung lượng đĩa
!df -h /home/ec2-user/SageMaker

In [ ]:
# Kiểm tra sử dụng GPU
!nvidia-smi

In [ ]:
# Dọn dẹp cache để tiết kiệm dung lượng
def cleanup_cache():
    !pip cache purge
    !rm -rf ~/.cache/huggingface/
    print("Đã dọn dẹp cache!")
    !df -h /home/ec2-user/SageMaker

# Uncomment dòng dưới để dọn dẹp cache
# cleanup_cache()

## 6. Tải thêm model (nếu cần)

In [ ]:
# Tải thêm model ControlNet
def download_controlnet_model(model_name, url):
    model_dir = "/home/ec2-user/SageMaker/stable-diffusion-webui/models/ControlNet"
    model_path = f"{model_dir}/{model_name}"
    
    if os.path.exists(model_path):
        print(f"Model {model_name} đã tồn tại.")
        return
    
    print(f"Đang tải {model_name}...")
    !wget -O {model_path} {url}
    print(f"Đã tải xong {model_name}!")

# Ví dụ: Tải model ControlNet Canny
# download_controlnet_model(
#     "control_v11p_sd15_canny.pth", 
#     "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth"
# )

## 7. Lưu ý quan trọng

1. **Tiết kiệm chi phí**: Nhớ dừng SageMaker Notebook Instance khi không sử dụng để tránh phát sinh chi phí không cần thiết.

2. **Lưu trữ models**: Các model được tải về sẽ bị mất khi instance bị xóa. Nếu cần, hãy sao lưu vào S3.

3. **Khởi động lại WebUI**: Nếu WebUI gặp lỗi, bạn có thể khởi động lại bằng cách chạy lại cell "Khởi động WebUI" ở trên.

4. **Tài nguyên GPU**: Nếu gặp lỗi CUDA out of memory, hãy thử thêm tham số `--medvram` hoặc `--lowvram` vào `COMMANDLINE_ARGS`.